# Climate and Soil

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:


Outputs:




In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder

# Load the categorical image and select the 'biome' band
biomes = ee.Image(f"{data_folder}/categorical").select("biome")
biomes_mask = biomes.eq(1).Or(biomes.eq(4)).rename("biome_mask")

## Soil Data from SoilGrids

250m resolution

        - Bulk Density
        - Cation Exchange Capacity
        - Clay Content
        - Coarse fragments (> 2 mm)
        - Nitrogen
        - Organic Carbon Density
        - Soil Organic Carbon Stock
        - pH
        - Sand Content
        - Soil Organic Carbon
    All averaged from 0-30cm depth and converted to the correct units.

In [2]:
bdod = ee.Image("projects/soilgrids-isric/bdod_mean").clip(roi) # bulk density
cec = ee.Image("projects/soilgrids-isric/cec_mean") # cation exchange capacity
clay = ee.Image("projects/soilgrids-isric/clay_mean").clip(roi)
cfvo = ee.Image("projects/soilgrids-isric/cfvo_mean").clip(roi) # coarse fragments
nitro = ee.Image("projects/soilgrids-isric/nitrogen_mean").clip(roi)
ocd = ee.Image("projects/soilgrids-isric/ocd_mean").clip(roi) # organic carbon density
ocs = ee.Image("projects/soilgrids-isric/ocs_mean").clip(roi) # soil organic carbon stock
phh2o = ee.Image("projects/soilgrids-isric/phh2o_mean").clip(roi)
sand = ee.Image("projects/soilgrids-isric/sand_mean").clip(roi)
soc = ee.Image("projects/soilgrids-isric/soc_mean").clip(roi) # soil organic carbon

# Function to select and calculate weighted mean for specific depth bands
def weighted_means(image):
    depths = ['0-5cm', '5-15cm', '15-30cm']
    weights = [1, 2, 3]    
    # Select the bands for each depth
    weighted_bands = image.select([f'.*_{depth}_mean' for depth in depths]).multiply(ee.Image.constant(weights))
    # Sum the weighted bands
    weighted_mean = weighted_bands.reduce(ee.Reducer.sum()).divide(6)
    return weighted_mean

# Unit conversions
bdod = bdod.multiply(10)  # cg/cm³ to kg/m³
nitro = nitro.divide(100)  # cg/kg to g/kg
soc = soc.divide(10)  # dg/kg to g/kg
cec = cec.divide(10)  # mmol(c)/kg to cmol/kg

# Convert nitrogen and soil organic carbon to g/m²
nitro = weighted_means(nitro.multiply(bdod)).rename("nitro")  # g/kg to g/m³
soc = weighted_means(soc.multiply(bdod)).rename("soc")  # g/kg to g/m³
cec = weighted_means(cec.multiply(bdod)).rename("cec")  # cmol/kg to cmol/kg

# Apply unit conversions and depth aggregation to other soil properties
clay = weighted_means(clay.divide(10)).rename("clay")  # g/kg (‰) to g/100g (%)
cfvo = weighted_means(cfvo.divide(10)).rename("cfvo")  # cm³/dm³ (‰) to cm³/100cm³ (%)
ocd = weighted_means(ocd.divide(10)).rename("ocd")  # hg/m³ to kg/m³
ocs = ocs.divide(10).rename("ocs")  # hg/m² to kg/m²
phh2o = weighted_means(phh2o.divide(10)).rename("phh2o")  # pH x 10 to pH
sand = weighted_means(sand.divide(10)).rename("sand")  # g/kg (‰) to g/100g (%)

# Combine all soil properties into a single image
soil_properties = cec.addBands([clay, cfvo, nitro, ocd, ocs, phh2o, sand, soc]).float()

# Export the final image
# export_image(soil_properties, "soilgrids", region = roi, scale = 250)

## CMIP6 Future projections

In [3]:
# get all images from CMIP6 folder

vars = ["moisture_in_upper_portion_of_soil_column",
        "near_surface_air_temperature",
        # "precipitation",
        # "near_surface_specific_humidity",
        "surface_downwelling_shortwave_radiation"]

scenarios = ["historical", "ssp126", "ssp245", "ssp585"]

abbreviations = ["musc", "nsat", "sdsr"]


# Map variable to abbreviation
var_to_abbrev = dict(zip(vars, abbreviations))

def make_scenario_raster(scenario):
    # Select the years
    years = range(1985, 2015) if scenario == "historical" else range(2015, 2050)
    
    # Load and rename all variable images for the scenario
    renamed_images = []
    for var in vars:
        abbrev = var_to_abbrev[var]
        image = ee.Image(f"{data_folder}/CMIP6/{var}_{scenario}")
        renamed = image.select(
            [f"b{i}" for i in range(1, len(years) + 1)],
            [f"{abbrev}_{year}" for year in years]
        )
        renamed_images.append(renamed)
    
    # Combine all variables into one multiband image
    combined_image = renamed_images[0]
    for img in renamed_images[1:]:
        combined_image = combined_image.addBands(img)
    
    return combined_image

# Example usage
ssp126 = make_scenario_raster("ssp126")
ssp245 = make_scenario_raster("ssp245")
ssp585 = make_scenario_raster("ssp585")
historical = make_scenario_raster("historical")

# export_image(ssp126, "CMIP6_ssp126", region = roi)
# export_image(ssp245, "CMIP6_ssp245", region = roi)
# export_image(ssp585, "CMIP6_ssp585", region = roi)
# export_image(historical, "CMIP6_historical", region = roi)


# map = geemap.Map()
# map.addLayer(ssp126, {}, "ssp126")
# map



## TerraClim Data

    Calculated yearly metrics.
        Summed (data shown as monthly totals):
        - Soil Moisture (mm)
        - Precipitation (mm)
        - Evapotranspiration (mm)
        - Climate Water Deficit (mm)

        Averaged (data shown as monthly averages):
        - Temperature (C)
        - Vapour Pressure Deficit (kPa)
        - Palmer Drought Severity Index (PDSI)

        Converted to kWh/m²/year (Total solar energy received per square meter over a year):
        - Solar Radiation (W/m^2)


https://gee-community-catalog.org/projects/terraclim/

<!-- # Terraclim
  https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
Bring temperature and precipitation and calculate seasonality -->

<!-- ## Seasonality index

Walsh and Lawler 1981 -->

In [ ]:
terraclim = (
    ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")
    .filterDate("1958-01-01", "2019-12-31")
    .select(["tmmx", "tmmn", "srad", "vpd", "soil", "aet", "pr", "def", "pdsi"])
    .map(lambda image: image.updateMask(biomes_mask))
)

# Function to calculate yearly metrics with scaling applied after filtering
def calculate_yearly_metrics(year):

    def scale_and_aggregate(var, reducer):
        data = terraclim.select(var).filter(ee.Filter.calendarRange(year, year, "year"))
        if var in ["tmmn", "tmmx", "soil", "aet", "def"]:
            data = data.map(lambda img: img.multiply(0.1))
        if var in ["pdsi", "vpd"]:
            data = data.map(lambda img: img.multiply(0.01))
        if var in ["tmmn", "tmmx", "pdsi", "vpd", "pr"]:
            data = data.reduce(reducer).float().rename(f"{var}_{year}")
        else:
            data = data.reduce(reducer).toInt16().rename(f"{var}_{year}")
        return data

    # Special function for solar radiation (convert W/m² to kWh/m²/year)
    def calculate_srad_annual(year):
        # Hours per month (non-leap year)
        hours_per_month = ee.List([744, 672, 744, 720, 744, 720, 744, 744, 720, 744, 720, 744])
        
        # Get monthly srad data for the year
        monthly_srad = terraclim.select("srad").filter(ee.Filter.calendarRange(year, year, "year"))
        
        # Convert to list and map over months
        monthly_list = monthly_srad.toList(12)
        
        def convert_monthly_to_energy(index):
            month_img = ee.Image(monthly_list.get(index))
            hours = ee.Number(hours_per_month.get(index))
            # Scale from W/m² to kWh/m²: multiply by 0.1 (TerraClimate scaling), then by hours, then divide by 1000
            return month_img.multiply(0.1).multiply(hours).divide(1000)
        
        # Convert each month and sum for annual total
        monthly_energy = ee.List.sequence(0, 11).map(convert_monthly_to_energy)
        annual_energy = ee.ImageCollection.fromImages(monthly_energy).sum()
        
        return annual_energy.float().rename(f"srad_{year}")
    

    # Define which variables are processed with sum or mean reducers
    sum_vars = ["soil", "pr", "aet", "def"]
    mean_vars = ["vpd", "pdsi"]


    # Aggregate sum variables (radiation, soil, precipitation)
    processed_vars = {var: scale_and_aggregate(var, ee.Reducer.sum()) for var in sum_vars}
    # Aggregate other mean variables
    processed_vars.update({var: scale_and_aggregate(var, ee.Reducer.mean()) for var in mean_vars})

    # Handle solar radiation separately
    processed_vars["srad"] = calculate_srad_annual(year)
    
    # Aggregate mean temperature (average of max and min)
    maxtemp = scale_and_aggregate("tmmx", ee.Reducer.mean())
    mintemp = scale_and_aggregate("tmmn", ee.Reducer.mean())
    processed_vars["temp"] = maxtemp.addBands(mintemp).reduce(ee.Reducer.mean()).float().rename(f"temp_{year}")

    return ee.Image.cat([*processed_vars.values()])

# Create a dictionary for variables with filtering and scaling applied after
vars = {var: terraclim.select(var) for var in ["tmmx", "tmmn", "srad", "vpd", "soil", "aet", "pr", "def", "pdsi"]}

# Calculate yearly metrics and combine into a single image
yearly_metrics = ee.Image.cat([calculate_yearly_metrics(year) for year in range(1958, 2020)])

# Function to calculate the mean for a given band pattern
def calculate_mean(var, new_name):
    year_bands = [f"{var}_{year}" for year in list(range(1985, 2020))]
    return yearly_metrics.select(year_bands).reduce(ee.Reducer.mean()).rename(new_name)

# Calculate the mean across all years for the desired variables
mean_metrics = {
    var: calculate_mean(var, f"mean_{var}")
    for var in ["pr", "srad", "temp", "vpd", "soil", "aet", "def", "pdsi"]
}

# Combine the mean layers into a single image
yearly_terraclim = ee.Image.cat([yearly_metrics, *mean_metrics.values()])

# Export the final image
# export_image(yearly_terraclim, "terraclim_1958_2019", region = roi, scale = 250)


### Climate
https://developers.google.com/earth-engine/datasets/catalog/NASA_GDDP-CMIP6

- huss: Near-surface relative humidity (%)
- pr: Mean of daily precipitation rate (kg m-2 s-1)
- rsds: Surface downwelling shortwave radiation (W m-2)
- tas: Daily near-surface air temperature (K)

In [ ]:
# incompleteSims=['BCC-CSM2-MR','CESM2', 'CESM2-WACCM','IITM-ESM','IPSL-CM6A-LR','KIOST-ESM',
    #   'MIROC6','NESM3','NorESM2-LM', 'TaiESM1']

climate_CMIP6 = (ee.ImageCollection("NASA/GDDP-CMIP6")
                     .map(lambda image: image.select(["hurs", "pr", "rsds", "tas"])))

def calculate_yearly_metrics(year):
    climate_year = climate_CMIP6.filterDate(f"{year}-01-01", f"{year}-12-31").mean()
    climate_year = climate_year.rename([f"{band}_{year}" for band in climate_year.bandNames().getInfo()])
    return climate_year

years = list(range(1985, 2051))

yearly_metrics = ee.Image.cat([calculate_yearly_metrics(year) for year in years])

# Calculate the mean across all years for the desired variables
mean_metrics = {name: calculate_mean(f".*{name}.*", f"mean_{name}") \
                for name in ["hurs", "pr", "rsds", "tas"]}

# Combine the mean layers into a single image
yearly_cmip6 = ee.Image.cat([yearly_metrics, *mean_metrics.values()])


export_image(yearly_cmip6, "yearly_cmip6", region = roi, scale = 10000)

# map = geemap.Map()
# map.addLayer(yearly_metrics.select("tas_1985"), {"min": 250, "max": 320, "palette": ["blue", "green", "yellow", "red"]}, "Temperature 1985")
# map